In [1]:
import pyspark
import json
import requests

In [2]:
sc = pyspark.SparkContext()
sqlCtx = pyspark.SQLContext(sc)

In [9]:
values = [
    ('HELLO', 'WORLD'),
    ('hello', 'world')
]

schema = ['firstWord', 'secondWord']

df = sqlCtx.createDataFrame(values, schema)

In [6]:
def insert_into_elastic(pandasDataFrame, index='tmp_index', typ = 'test', server ='elasticsearch', chunk_size = 2000):
    headers = {'content-type': 'application/x-ndjson', 'Accept-Charset': 'UTF-8'}
    dataFrameRowsJson =[v.dropna().to_json() for k,v in pandasDataFrame.iterrows()]
    rowsToSend = ['{ "index" : { "_index" : "%s", "_type" : "%s"} }\n%s' % (index, typ, rowJson) 
               for rowJson in dataFrameRowsJson]
    i=0
    cuenta_errores=0
    while i<len(rowsToSend):
        serverAPI = 'http://'+server+':9200/_bulk/' 
        data='\n'.join(rowsToSend[i:min([i+chunk_size,len(rowsToSend)])])
        data = data + '\n'
        r = requests.post(serverAPI, data=data, headers=headers)
        i = i + chunk_size
        r = json.loads(r.content)
        for item in r['items']:
            if item['index']['status']!=201:
                #print item['index']
                #logger.info('Error en la insercción de ELASTICSEARCH: '+ str(item['index']))
                cuenta_errores=cuenta_errores+1
    return 'Number of errors: %s' % cuenta_errores

In [10]:
insert_into_elastic(df.toPandas(),index='test_index', server = 'elasticsearch', chunk_size = 2000)

'Number of errors: 0'